# Data Modelling

The present section concerns itself with the steps needed to create a datastore from the raw dataset that we intend to use during the course of our study. It is the first step which we need to take in order to curate the data that we can use for our study.

## Step 1: Downloading the data

For the purpose of our study we intend to use [Yelp Academic Dataset], for more detail overview of the dataset please refer to the official [documentation]. The dataset can be downloaded after submitting some basic details on the [download] page. The dataset comes in 6 a json files with information pertainin to 6 different enteties i.e. `business`, `user`, `review`, `tips`, `check-ins` and `photos`. But for the purpose of our study we will limit our focus to 3 enteties i.e. `business`, `user` and `review`. The raw json files for the correspoding entities can be located [here].


[Yelp Academic Dataset]: https://www.yelp.com/dataset
[documentation]: https://www.yelp.com/dataset/documentation/main
[download]: https://www.yelp.com/dataset/download
[here]: ./data/raw/json

## Step 2: JSON to CSV

As mentioned the dataset which we intend to use is in json format. Hence, we need to convert it into csv format so that further analysis becomes easier. Hence, below we convert raw json files to csv using pandas.

In [ ]:
import pandas as pd

# reading reviews json file into pandas dataframe and saving it as csv
df_reviews = pd.read_json('./data/raw/json/yelp_academic_dataset_review.json', lines=True)
df_reviews.to_csv('./data/raw/csv/yelp_academic_dataset_review.csv', index=False)

# reading users json file into pandas dataframe and saving it as csv
df_users = pd.read_json('./data/raw/json/yelp_academic_dataset_user.json', lines=True)
df_users.to_csv('./data/raw/csv/yelp_academic_dataset_user.csv', index=False)

# reading businesses json file into pandas dataframe and saving it as csv
df_businesses = pd.read_json('./data/raw/json/yelp_academic_dataset_business.json', lines=True)
df_businesses.to_csv('./data/raw/csv/yelp_academic_dataset_business.csv', index=False)

## Step 3: CSV to SQLite Database

To make the further operating with the dataset easier and allow for complecated operation such as joins which be quite difficult to perform on raw huge csv file we decided to create a in memory database out of the csv using SQLite. Using followig commands on command line we create a database named `yelp_db` and then import our three csvs created from above step as three tables in our database i.e. `businesses`, `users` and `reviews`.

```console
bash> sqlite3 yelp_db.sqlite
sqlite> .mode csv
sqlite> .import ./data/raw/csv/yelp_academic_dataset_business.csv businesses
sqlite> .import ./data/raw/csv/yelp_academic_dataset_user.csv users
sqlite> .import ./data/raw/csv/yelp_academic_dataset_review.csv reviews
sqlite> .exit
```

## Step 4: Leaveraing database functionalities

After importing our csvs to the database we make certain adjustments to our tables.

- **businesses:** We make `business_id` as the primary key and change data types of columns `stars`, `review_count` and `is_open` as `REAL`, `INT` and `INT` respectively by running following commands in SQLite commandline.

        create table businesses_dg_tmp
        (
            business_id  TEXT
                constraint businesses_pk
                    primary key,
            name         TEXT,
            address      TEXT,
            city         TEXT,
            state        TEXT,
            postal_code  TEXT,
            latitude     TEXT,
            longitude    TEXT,
            stars        REAL,
            review_count INTEGER,
            is_open      INT,
            attributes   TEXT,
            categories   TEXT,
            hours        TEXT
        );

        insert into businesses_dg_tmp(business_id, name, address, city, state,
                                      postal_code, latitude, longitude, stars,
                                      review_count, is_open, attributes, categories,
                                      hours)
        select business_id,
               name,
               address,
               city,
               state,
               postal_code,
               latitude,
               longitude,
               stars,
               review_count,
               is_open,
               attributes,
               categories,
               hours
        from businesses;

        drop table businesses;

        alter table businesses_dg_tmp
            rename to businesses;
- **users:** We make `user_id` as the primary key and change the datatypes of columns `review_count`, `useful`, `funny`, `cool`, `fans`, `average_stars`, `compliment_hot`, `compliment_more`, `compliment_profile`, `compliment_cute`, `compliment_list`, `compliment_note`, `compliment_plain`, `compliment_cool`, `compliment_funny`, `compliment_writer` and `compliment_photos` using following commands in SQLite commandline.

        CREATE TABLE users_dg_tmp
          (
             user_id            TEXT PRIMARY KEY,
             name               TEXT,
             review_count       INTEGER,
             yelping_since      TEXT,
             useful             INTEGER,
             funny              INTEGER,
             cool               INTEGER,
             elite              TEXT,
             friends            TEXT,
             fans               INTEGER,
             average_stars      REAL,
             compliment_hot     INTEGER,
             compliment_more    INTEGER,
             compliment_profile INTEGER,
             compliment_cute    INTEGER,
             compliment_list    INTEGER,
             compliment_note    INTEGER,
             compliment_plain   INTEGER,
             compliment_cool    INTEGER,
             compliment_funny   INTEGER,
             compliment_writer  INTEGER,
             compliment_photos  INTEGER
          );

        INSERT INTO users_dg_tmp
                    (user_id,
                     name,
                     review_count,
                     yelping_since,
                     useful,
                     funny,
                     cool,
                     elite,
                     friends,
                     fans,
                     average_stars,
                     compliment_hot,
                     compliment_more,
                     compliment_profile,
                     compliment_cute,
                     compliment_list,
                     compliment_note,
                     compliment_plain,
                     compliment_cool,
                     compliment_funny,
                     compliment_writer,
                     compliment_photos)
        SELECT user_id,
               name,
               review_count,
               yelping_since,
               useful,
               funny,
               cool,
               elite,
               friends,
               fans,
               average_stars,
               compliment_hot,
               compliment_more,
               compliment_profile,
               compliment_cute,
               compliment_list,
               compliment_note,
               compliment_plain,
               compliment_cool,
               compliment_funny,
               compliment_writer,
               compliment_photos
        FROM   users;

        DROP TABLE users;

        ALTER TABLE users_dg_tmp
          RENAME TO users;


- **reviews:** We make `review_id` as primary key and `user_id` and `business_id` as foreign key pointing to `users` and `businesses`. Moreover, we chance the type of `stars`, `useful`, `funny` and `cool` columns to `INT` using the following commands in SQLite commandline.

        CREATE TABLE reviews_dg_tmp
          (
             review_id   TEXT PRIMARY KEY,
             user_id     TEXT CONSTRAINT reviews_users_user_id_fk REFERENCES users,
             business_id TEXT CONSTRAINT reviews_businesses_business_id_fk REFERENCES
             businesses,
             stars       INTEGER,
             useful      INTEGER,
             funny       INTEGER,
             cool        INTEGER,
             text        TEXT,
             DATE        TEXT
          );

        INSERT INTO reviews_dg_tmp
                    (review_id,
                     user_id,
                     business_id,
                     stars,
                     useful,
                     funny,
                     cool,
                     text,
                     DATE)
        SELECT review_id,
               user_id,
               business_id,
               stars,
               useful,
               funny,
               cool,
               text,
               DATE
        FROM   reviews;

        DROP TABLE reviews;

        ALTER TABLE reviews_dg_tmp
          RENAME TO reviews;

## Step 5: Restaurant Data

In this study we intend to we intend to cover data corresponding to to businesses which are restaurants. Hence we take following steps to extract restaurant data out of the complete data

- **Extracting Restaurants from businesses:** We being by extracting `businesses` which have `restaurants` in their `categories` columsn and save them to a new table names `restaurants` using below command in SQLite commandline.

        CREATE TABLE restaurants AS
          SELECT *
          FROM   businesses
          WHERE  Lower(categories) LIKE '%restaurants%';
- **Extracting restaurant reviews:** Using the newly created `restaurants` table we extract reviews that belong to restaurants and save them in a new table named `restaurant_reviews` using following command in SQLite commandline.

        create table restaurant_reviews as
        select *
        from reviews
        where reviews.business_id in (select business_id from restaurants);
- **Extracting users corresponding to reviews:** Using the newly created `restaurant_reviews` table we extract the users who posted those reviews and save them in a new table named `restaurant_users` using the following command in SQLite commandline.

        create table restaurant_users as
        select *
        from users
        where users.user_id in (select user_id from restaurant_reviews);
- **Recreating Linkages:** As the tables we extracted does not consits of the primary key and foreign key linkages we recreate the linkages using following command in SQLite commandline
-- **Restaurants Primary Key:**


        CREATE TABLE restaurants_dg_tmp
          (
             business_id  TEXT PRIMARY KEY,
             name         TEXT,
             address      TEXT,
             city         TEXT,
             state        TEXT,
             postal_code  TEXT,
             latitude     TEXT,
             longitude    TEXT,
             stars        REAL,
             review_count INT,
             is_open      INT,
             attributes   TEXT,
             categories   TEXT,
             hours        TEXT
          );

        INSERT INTO restaurants_dg_tmp
                    (business_id,
                     name,
                     address,
                     city,
                     state,
                     postal_code,
                     latitude,
                     longitude,
                     stars,
                     review_count,
                     is_open,
                     attributes,
                     categories,
                     hours)
        SELECT business_id,
               name,
               address,
               city,
               state,
               postal_code,
               latitude,
               longitude,
               stars,
               review_count,
               is_open,
               attributes,
               categories,
               hours
        FROM   restaurants;

        DROP TABLE restaurants;

        ALTER TABLE restaurants_dg_tmp
          RENAME TO restaurants;

    -- **Restaurant user Primary Key:**

        CREATE TABLE restaurant_users_dg_tmp
          (
             user_id            TEXT PRIMARY KEY,
             name               TEXT,
             review_count       INT,
             yelping_since      TEXT,
             useful             INT,
             funny              INT,
             cool               INT,
             elite              TEXT,
             friends            TEXT,
             fans               INT,
             average_stars      REAL,
             compliment_hot     INT,
             compliment_more    INT,
             compliment_profile INT,
             compliment_cute    INT,
             compliment_list    INT,
             compliment_note    INT,
             compliment_plain   INT,
             compliment_cool    INT,
             compliment_funny   INT,
             compliment_writer  INT,
             compliment_photos  INT
          );

        INSERT INTO restaurant_users_dg_tmp
                    (user_id,
                     name,
                     review_count,
                     yelping_since,
                     useful,
                     funny,
                     cool,
                     elite,
                     friends,
                     fans,
                     average_stars,
                     compliment_hot,
                     compliment_more,
                     compliment_profile,
                     compliment_cute,
                     compliment_list,
                     compliment_note,
                     compliment_plain,
                     compliment_cool,
                     compliment_funny,
                     compliment_writer,
                     compliment_photos)
        SELECT user_id,
               name,
               review_count,
               yelping_since,
               useful,
               funny,
               cool,
               elite,
               friends,
               fans,
               average_stars,
               compliment_hot,
               compliment_more,
               compliment_profile,
               compliment_cute,
               compliment_list,
               compliment_note,
               compliment_plain,
               compliment_cool,
               compliment_funny,
               compliment_writer,
               compliment_photos
        FROM   restaurant_users;

        DROP TABLE restaurant_users;

        ALTER TABLE restaurant_users_dg_tmp
          RENAME TO restaurant_users;

    -- **Restaurant review Primary Key and Foreign Keys:**

        create table restaurant_reviews_dg_tmp
        (
            review_id   TEXT
                primary key,
            user_id     TEXT
                constraint restaurant_reviews_restaurant_users_user_id_fk
                    references restaurant_users,
            business_id TEXT
                constraint restaurant_reviews_restaurants_business_id_fk
                    references restaurants,
            stars       INT,
            useful      INT,
            funny       INT,
            cool        INT,
            text        TEXT,
            date        TEXT
        );

        insert into restaurant_reviews_dg_tmp(review_id, user_id, business_id, stars,
                                              useful, funny, cool, text, date)
        select review_id,
               user_id,
               business_id,
               stars,
               useful,
               funny,
               cool,
               text,
               date
        from restaurant_reviews;

        drop table restaurant_reviews;

        alter table restaurant_reviews_dg_tmp
            rename to restaurant_reviews;

    -- **Restaurant reviews Index on user_id:**

        create index restaurant_reviews_user_id_index
            on restaurant_reviews (user_id);